In [1]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

class AdversarialAutoencoder():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 10

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the encoder / decoder
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

        img = Input(shape=self.img_shape)
        # The generator takes the image, encodes it and reconstructs it
        # from the encoding
        encoded_repr = self.encoder(img)
        reconstructed_img = self.decoder(encoded_repr)

        # For the adversarial_autoencoder model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator determines validity of the encoding
        validity = self.discriminator(encoded_repr)

        # The adversarial_autoencoder model  (stacked generator and discriminator)
        self.adversarial_autoencoder = Model(img, [reconstructed_img, validity])
        self.adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
            loss_weights=[0.999, 0.001],
            optimizer=optimizer)


    def build_encoder(self):
        # Encoder

        img = Input(shape=self.img_shape)

        h = Flatten()(img)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        latent_repr = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2), lambda p: p[0])([mu, log_var])
        return Model(img, latent_repr)

    def build_decoder(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        img = model(z)

        return Model(z, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation="sigmoid"))
        model.summary()

        encoded_repr = Input(shape=(self.latent_dim, ))
        validity = model(encoded_repr)

        return Model(encoded_repr, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            latent_fake = self.encoder.predict(imgs)
            latent_real = np.random.normal(size=(batch_size, self.latent_dim))

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(latent_real, valid)
            d_loss_fake = self.discriminator.train_on_batch(latent_fake, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5

        z = np.random.normal(size=(r*c, self.latent_dim))
        gen_imgs = self.decoder.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "aae_generator")
        save(self.discriminator, "aae_discriminator")



if __name__ == '__main__':
    aae = AdversarialAutoencoder()
    aae.train(epochs=20000, batch_size=32, sample_interval=200)

C:\Users\HoJun\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

C:\Users\HoJun\Anaconda3\envs\py35\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.733556, acc: 26.56%] [G loss: 0.937035, mse: 0.937288]
1 [D loss: 0.707623, acc: 42.19%] [G loss: 0.916056, mse: 0.916295]
2 [D loss: 0.680825, acc: 56.25%] [G loss: 0.905700, mse: 0.905853]
3 [D loss: 0.576342, acc: 90.62%] [G loss: 0.851395, mse: 0.851151]
4 [D loss: 0.427631, acc: 89.06%] [G loss: 0.770913, mse: 0.769752]
5 [D loss: 0.335212, acc: 85.94%] [G loss: 0.676940, mse: 0.674328]
6 [D loss: 0.319021, acc: 84.38%] [G loss: 0.558585, mse: 0.554285]
7 [D loss: 0.280114, acc: 93.75%] [G loss: 0.423230, mse: 0.417027]
8 [D loss: 0.264543, acc: 90.62%] [G loss: 0.336298, mse: 0.329062]
9 [D loss: 0.228792, acc: 98.44%] [G loss: 0.309931, mse: 0.301974]
10 [D loss: 0.252725, acc: 95.31%] [G loss: 0.311988, mse: 0.304262]
11 [D loss: 0.210610, acc: 98.44%] [G loss: 0.261670, mse: 0.254204]
12 [D loss: 0.215971, acc: 98.44%] [G loss: 0.277609, mse: 0.270791]
13 [D loss: 0.223279, acc: 98.44%] [G loss: 0.306128, mse: 0.300226]
14 [D loss: 0.186962, acc: 98.44%] [G loss: 

119 [D loss: 0.276534, acc: 85.94%] [G loss: 0.143574, mse: 0.138189]
120 [D loss: 0.438229, acc: 81.25%] [G loss: 0.155118, mse: 0.150654]
121 [D loss: 0.282320, acc: 84.38%] [G loss: 0.165709, mse: 0.157210]
122 [D loss: 0.194385, acc: 92.19%] [G loss: 0.151019, mse: 0.145782]
123 [D loss: 0.384230, acc: 82.81%] [G loss: 0.154893, mse: 0.150129]
124 [D loss: 0.213180, acc: 87.50%] [G loss: 0.156693, mse: 0.149042]
125 [D loss: 0.235976, acc: 89.06%] [G loss: 0.156228, mse: 0.149507]
126 [D loss: 0.238015, acc: 87.50%] [G loss: 0.149800, mse: 0.143218]
127 [D loss: 0.235959, acc: 85.94%] [G loss: 0.138563, mse: 0.131558]
128 [D loss: 0.228632, acc: 89.06%] [G loss: 0.148293, mse: 0.142829]
129 [D loss: 0.197055, acc: 93.75%] [G loss: 0.142235, mse: 0.137069]
130 [D loss: 0.188964, acc: 93.75%] [G loss: 0.161058, mse: 0.155590]
131 [D loss: 0.215980, acc: 92.19%] [G loss: 0.140639, mse: 0.136148]
132 [D loss: 0.170558, acc: 93.75%] [G loss: 0.141105, mse: 0.134466]
133 [D loss: 0.20704

...

19517 [D loss: 0.305453, acc: 87.50%] [G loss: 0.060470, mse: 0.058285]
19518 [D loss: 0.457870, acc: 84.38%] [G loss: 0.053744, mse: 0.051813]
19519 [D loss: 0.364225, acc: 79.69%] [G loss: 0.052871, mse: 0.050938]
19520 [D loss: 0.402176, acc: 79.69%] [G loss: 0.044785, mse: 0.042852]
19521 [D loss: 0.405554, acc: 85.94%] [G loss: 0.062772, mse: 0.061071]
19522 [D loss: 0.340491, acc: 85.94%] [G loss: 0.050669, mse: 0.048732]
19523 [D loss: 0.330306, acc: 84.38%] [G loss: 0.049797, mse: 0.047542]
19524 [D loss: 0.379227, acc: 84.38%] [G loss: 0.057206, mse: 0.054930]
19525 [D loss: 0.383466, acc: 85.94%] [G loss: 0.059103, mse: 0.057182]
19526 [D loss: 0.461993, acc: 75.00%] [G loss: 0.066415, mse: 0.064579]
19527 [D loss: 0.409488, acc: 78.12%] [G loss: 0.053277, mse: 0.051427]
19528 [D loss: 0.296403, acc: 89.06%] [G loss: 0.041208, mse: 0.039100]
19529 [D loss: 0.386077, acc: 79.69%] [G loss: 0.054522, mse: 0.052691]
19530 [D loss: 0.343279, acc: 87.50%] [G loss: 0.051867, mse: 0.

19633 [D loss: 0.283703, acc: 85.94%] [G loss: 0.061268, mse: 0.059042]
19634 [D loss: 0.423174, acc: 79.69%] [G loss: 0.066313, mse: 0.064530]
19635 [D loss: 0.395200, acc: 79.69%] [G loss: 0.048894, mse: 0.047118]
19636 [D loss: 0.352931, acc: 84.38%] [G loss: 0.054387, mse: 0.052186]
19637 [D loss: 0.377934, acc: 81.25%] [G loss: 0.057472, mse: 0.055514]
19638 [D loss: 0.563621, acc: 71.88%] [G loss: 0.048459, mse: 0.046742]
19639 [D loss: 0.495196, acc: 82.81%] [G loss: 0.044637, mse: 0.043053]
19640 [D loss: 0.387489, acc: 87.50%] [G loss: 0.055936, mse: 0.054070]
19641 [D loss: 0.361219, acc: 85.94%] [G loss: 0.051765, mse: 0.050062]
19642 [D loss: 0.344485, acc: 82.81%] [G loss: 0.056690, mse: 0.054463]
19643 [D loss: 0.402891, acc: 82.81%] [G loss: 0.062758, mse: 0.060725]
19644 [D loss: 0.404641, acc: 84.38%] [G loss: 0.055824, mse: 0.053620]
19645 [D loss: 0.360877, acc: 82.81%] [G loss: 0.051171, mse: 0.049326]
19646 [D loss: 0.504803, acc: 75.00%] [G loss: 0.066923, mse: 0.

19747 [D loss: 0.336768, acc: 85.94%] [G loss: 0.056845, mse: 0.054950]
19748 [D loss: 0.400516, acc: 81.25%] [G loss: 0.062093, mse: 0.060106]
19749 [D loss: 0.362214, acc: 84.38%] [G loss: 0.055030, mse: 0.053134]
19750 [D loss: 0.469001, acc: 75.00%] [G loss: 0.055736, mse: 0.054162]
19751 [D loss: 0.429343, acc: 81.25%] [G loss: 0.046735, mse: 0.044861]
19752 [D loss: 0.497317, acc: 78.12%] [G loss: 0.064912, mse: 0.063452]
19753 [D loss: 0.353485, acc: 84.38%] [G loss: 0.049068, mse: 0.046960]
19754 [D loss: 0.324719, acc: 87.50%] [G loss: 0.059453, mse: 0.057475]
19755 [D loss: 0.501281, acc: 76.56%] [G loss: 0.061775, mse: 0.060073]
19756 [D loss: 0.370147, acc: 84.38%] [G loss: 0.064081, mse: 0.062233]
19757 [D loss: 0.434812, acc: 85.94%] [G loss: 0.054407, mse: 0.052495]
19758 [D loss: 0.433854, acc: 81.25%] [G loss: 0.052731, mse: 0.050788]
19759 [D loss: 0.331259, acc: 85.94%] [G loss: 0.051632, mse: 0.049241]
19760 [D loss: 0.349920, acc: 87.50%] [G loss: 0.057092, mse: 0.

19863 [D loss: 0.511777, acc: 82.81%] [G loss: 0.061227, mse: 0.059454]
19864 [D loss: 0.470438, acc: 78.12%] [G loss: 0.066586, mse: 0.064664]
19865 [D loss: 0.468597, acc: 73.44%] [G loss: 0.055206, mse: 0.053002]
19866 [D loss: 0.340308, acc: 90.62%] [G loss: 0.046810, mse: 0.044847]
19867 [D loss: 0.415926, acc: 82.81%] [G loss: 0.053644, mse: 0.052074]
19868 [D loss: 0.416707, acc: 81.25%] [G loss: 0.049236, mse: 0.047220]
19869 [D loss: 0.478189, acc: 76.56%] [G loss: 0.058327, mse: 0.056729]
19870 [D loss: 0.459839, acc: 79.69%] [G loss: 0.059216, mse: 0.057346]
19871 [D loss: 0.422633, acc: 81.25%] [G loss: 0.061739, mse: 0.060097]
19872 [D loss: 0.354564, acc: 85.94%] [G loss: 0.064373, mse: 0.062361]
19873 [D loss: 0.470237, acc: 81.25%] [G loss: 0.049193, mse: 0.047023]
19874 [D loss: 0.294385, acc: 89.06%] [G loss: 0.050714, mse: 0.048725]
19875 [D loss: 0.420084, acc: 76.56%] [G loss: 0.061507, mse: 0.059455]
19876 [D loss: 0.348033, acc: 85.94%] [G loss: 0.049439, mse: 0.

19977 [D loss: 0.383320, acc: 81.25%] [G loss: 0.055794, mse: 0.053882]
19978 [D loss: 0.370752, acc: 85.94%] [G loss: 0.052195, mse: 0.050237]
19979 [D loss: 0.367932, acc: 82.81%] [G loss: 0.060207, mse: 0.058437]
19980 [D loss: 0.363466, acc: 82.81%] [G loss: 0.054303, mse: 0.052328]
19981 [D loss: 0.434612, acc: 84.38%] [G loss: 0.053089, mse: 0.051314]
19982 [D loss: 0.311262, acc: 89.06%] [G loss: 0.052155, mse: 0.050166]
19983 [D loss: 0.575867, acc: 75.00%] [G loss: 0.064709, mse: 0.062905]
19984 [D loss: 0.365544, acc: 85.94%] [G loss: 0.054976, mse: 0.052816]
19985 [D loss: 0.452444, acc: 78.12%] [G loss: 0.051346, mse: 0.049750]
19986 [D loss: 0.421387, acc: 82.81%] [G loss: 0.051871, mse: 0.050016]
19987 [D loss: 0.419524, acc: 75.00%] [G loss: 0.059262, mse: 0.057552]
19988 [D loss: 0.500902, acc: 82.81%] [G loss: 0.056964, mse: 0.055362]
19989 [D loss: 0.291077, acc: 89.06%] [G loss: 0.058299, mse: 0.056060]
19990 [D loss: 0.356290, acc: 79.69%] [G loss: 0.062924, mse: 0.

In [3]:
aae.save_model().save()

AttributeError: 'AdversarialAutoencoder' object has no attribute 'generator'